In [1]:
import os
import sys

In [2]:
os.chdir('..')

# dbest-isi/searchless-chess-9M-selfplay
https://huggingface.co/dbest-isi/searchless-chess-9M-selfplay

In [3]:
!ls src

__pycache__	 distill_model.py	      load_model.ipynb
data_numpy.py	 distilled_student_model.pth  model_wrapper.py
data_pytorch.py  eda.ipynb		      save_data_to_np.py
data_size.py	 lightning_logs		      transformer_pytorch.py


In [11]:
from src.model_wrapper import create_predict_func
from huggingface_hub import snapshot_download

In [8]:
model_path = snapshot_download(
    repo_id="dbest-isi/searchless-chess-9M-selfplay",
    local_dir="./searchless_chess_model"
)
print(f"Model downloaded to: {model_path}")
model_path = '/teamspace/studios/this_studio/distill-chess/searchless_chess_model'

In [9]:
# Add bundled code to Python path
sys.path.insert(0, f"{model_path}/searchless_chess_code")

In [12]:

import hf_model
import utils
import tokenizer

# Load the model
model = hf_model.SearchlessChessModel.from_pretrained(model_path)
model.predict = create_predict_func(model, utils, tokenizer)

# Make a prediction
fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1"
result = model.predict(fen, temperature=1.0)

print(f"Best move: {result['best_move']}")
print(f"Q-value: {result['q_value']:.4f}")
print(f"Action probabilities shape: {result['action_probs'].shape}")

Best move: b7b5
Q-value: 0.7652
Action probabilities shape: (1968,)


In [7]:
import sys
from huggingface_hub import snapshot_download

# Download model from HuggingFace Hub
# model_path = snapshot_download(
#     repo_id="dbest-isi/searchless-chess-9M-selfplay",
#     local_dir="./searchless_chess_model"
# )

# Add bundled code to Python path
sys.path.insert(0, f"{model_path}/searchless_chess_code")

# Import model wrapper
import hf_model

# Load the model
model = hf_model.SearchlessChessModel.from_pretrained(model_path)

# Make a prediction
fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1"
result = model.predict(fen, temperature=1.0)

print(f"Best move: {result['best_move']}")
print(f"Q-value: {result['q_value']:.4f}")
print(f"Action probabilities shape: {result['action_probs'].shape}")


ModuleNotFoundError: No module named 'hf_model'

In [13]:
!ls ./searchless_chess_model/searchless_chess_code

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


__init__.py  config.py	   hf_model.py	 transformer.py
__pycache__  constants.py  tokenizer.py  utils.py


In [14]:
from huggingface_hub import snapshot_download
model_path = snapshot_download(
    repo_id="dbest-isi/searchless-chess-9M-selfplay",
    local_dir="./searchless_chess_model",
    local_dir_use_symlinks=False,
    revision="main"
)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

In [8]:
searchless_chess_code/
params/
model_config.json


{'q_value': 0.5073606371879578,
 'action_probs': array([0.4875268 , 0.51247317], dtype=float32),
 'best_action': 1,
 'best_move': 'a1c1'}

# Load chessformer model

In [7]:
path = os.path.abspath('../chessformers')
# sys.path.insert(0, path)
os.chdir(path)

In [8]:
import torch
import chess
from chessformers.model import Transformer # Import the model class
from chessformers.tokenizer import Tokenizer # <-- 1. Import the Tokenizer

# --- 1. Define Model Parameters ---
# These parameters MUST match the ones used to train the saved model.
d_model = 512
n_head = 8
n_layers = 8
d_k = 64
d_v = 64
d_ff = 2048
dropout = 0.1
max_len = 128  # Max sequence length

# --- 2. Instantiate Tokenizer and Model ---
# We need the tokenizer to get vocab size and padding id
tokenizer = Tokenizer() # <-- 2. Create an instance of the tokenizer

print("Initializing model architecture...")
model = Transformer(
    n_token=tokenizer.vocab_size, # <-- 3. Add n_token
    pad_idx=tokenizer.pad_id,   # <-- 4. Add pad_idx
    n_layers=n_layers,
    n_head=n_head,
    d_model=d_model,
    d_k=d_k,
    d_v=d_v,
    d_ff=d_ff,
    dropout=dropout,
    max_len=max_len
)

# --- 3. Load the Pretrained Weights ---
weights_path = './model/chessformer_epoch_13.pth'

try:
    print(f"Loading pretrained weights from {weights_path}...")
    state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    
    # --- 4. Set Model to Evaluation Mode ---
    model.eval()
    
    print("\nSuccessfully loaded pretrained model!")
    print(f"Model has {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

except FileNotFoundError:
    print(f"Error: Weights file not found at {weights_path}")
    print("Please download 'chessformer_epoch_13.pth' and place it in the './model/' directory.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

Initializing model architecture...


AttributeError: 'Tokenizer' object has no attribute 'pad_id'

In [21]:
import torch
import chess
from chessformers.model import Transformer # Import the model class
from chessformers.tokenizer import Tokenizer # Import the Tokenizer

# --- 1. Define Model Parameters ---
d_model = 512
n_head = 8
n_layers = 8
d_k = 64
d_v = 64
d_ff = 2048
dropout = 0.1
max_len = 128  # Max sequence length

# --- 2. Instantiate Tokenizer and Model ---
tokenizer = Tokenizer()

print("Initializing model architecture...")
model = Transformer(
    tokenizer=tokenizer,
    num_tokens=tokenizer.vocab_size,   
    num_layers=n_layers,
    num_heads=n_head,
    dim_model=d_model,
    # d_k=d_k,
    # d_v=d_v,
    # d_ff=d_ff,
    dropout_p=dropout,
    # max_len=max_len
)

# --- 3. Load the Pretrained Weights ---
weights_path = './model/chessformer_epoch_13.pth'

try:
    print(f"Loading pretrained weights from {weights_path}...")
    state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    
    # --- 4. Set Model to Evaluation Mode ---
    model.eval()
    
    print("\nSuccessfully loaded pretrained model!")
    print(f"Model has {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

except FileNotFoundError:
    print(f"Error: Weights file not found at {weights_path}")
    print("Please download 'chessformer_epoch_13.pth' and place it in the './model/' directory.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

Initializing model architecture...


TypeError: Transformer.__init__() missing 3 required positional arguments: 'tokenizer', 'd_hid', and 'n_positions'

In [15]:
Transformer()

TypeError: Transformer.__init__() missing 8 required positional arguments: 'tokenizer', 'num_tokens', 'dim_model', 'num_heads', 'd_hid', 'num_layers', 'dropout_p', and 'n_positions'